## Introduction
Wees goed voor de bossen, niet alleen in de natuur, maar ook voor het oplossen van problemen.

*Random Forests* is een van de meest versatiele machine learning algoritmes die vandaag-de-dag beschikbaar zijn. Door gebruik van hun ingebouwde 'ensembling' (samenstelling) vermogen, is het zelf nog makkelijker geworden om ze te gebruiken om een generiek model te maken (op welke data set dan ook). 

Echter, vaak worden Random Forest slechts als blackbox model toegpast, zonder dat de programmeurs precies weten wat ze precies aan het doen zijn. Het makkelijkste onderdeel van Machine Learning is het coden van de oplossing; begrijpen wat er gebeurt vergt echter wat meer moeite.

Random Forests worden nogal eens verward met 'bagging'; we zullen hieronder het verschil proberen uit te leggen. 

## Voor- en nadelen
Random Forests zijn een uitbereiding op Decision Trees, die reeds behandeld zijn bij Computational Modelling.

De voordelen van Random Forests zijn de volgenden:
* Random Forest kunnen zowel gebruikt worden voor classificatie en regressie problemen, en doet het op beide vlakken een redelijk goed;
* Een van de voordelen van Random Forests is de kracht die ze heeft in het  gebruik van grote data set met hoge dimensionaliteit. RF kan eenvoudig omgaan met duizenden input variabelen en identificeert zelfstandig de belangrijkste variabelen waardoor het ook meteen als Dimension Reduction Mehtod kan worden beschouwd;
* RF heeft een effectieve methode voor het omgaan met missende data en behoudt accuraatheid zelfs als grote delen van de data niet beschikbaar zijn;
* RF heeft methoden om fouten in de balans tussen klassen te corrigeren;

Nadelen van random forests:
* RFs zijn prima voor classificatie, maar minder geschikt voor regressie problemen, omdat het geen continue precieze voorspelling kan doen. In het geval van regressie is RF beperkt tot het bereik van de training data. Tevens is het erg gevoelig voor overfitting als de training data erg noisy is.
* RF wordt vaak gezien als blackbox methode, met weinig controle over wat het model precies doet. Het beste is om verschillende combinaties van parameters en random seeds te proberen.
 

## Wat is een Random Forest?
Random Forest is een tree-gebaseerd algoritme waarbij verschillende bomen (decision trees) worden gemaakt en worden gecombineerd zodat ze een verbetering leveren voor de generalisatie van het model. De methode waarmee bomen met elkaar worden gecombineerd heet de 'ensemble' methode. Ensembling is niets anders dan het combineren van zwakke modellen (individuele bomen) tot een sterk model.

Zeg, bijvoorbeeld, dat je een film wilt gaan kijken, maar je bent onzeker over de reviews van de film. Je vraagt aan 10 personen die de film hebben gekeken, wat ze ervan vonden. 8 van hen vertellen je dat "de film fantastisch is". Aangezien de meerderheid lovend is over de film, besluit je om toch te gaan. Dit is een voorbeeld van ensemble in het dagelijkse leven.

**Trivia**: Het random forests algoritme is gemaakt in 2001 door Leo Brieman en Adele Cutler.

## Hoe werkt het precies? (Decision trees)
Om te begrijpen hoe een random forest precies werkt, moeten we eerst even bespreken hoe een decision tree if werkt:

![decision tree](http://blog.hackerearth.com/wp-content/uploads/2016/12/root-01.jpg)

1. Een decision tree verdeelt een gegeven een data frame (n $\times$ p) gebaseerd op regels (if-then). Deze regels verdelen de datapunten in verschillende, niet overlappende gebieden. Deze regels worden bepaald aan de hand van hun bijdrage aan de homogeenheid of puurheid van de resulterende kind-knopen van de boom (X2, X3)

2. In het plaatje hierboven wordt de variabele X1 gekozen omdat deze de hoogste puurheid levert in de kind-knopen, waardoor ze de root-node wordt. Een variabele in de root-node wordt gezien als de meest belangrijke (voorspellende) variabele in de data set.

3. Maar hoe bepalen we nu precies die puurheid? Of in andere woorden, hoe bepaalt de boom op welke variabele het gaat splitsen?

In regressie bomen (waar de output een voorspelling op basis van het gemiddelde van de observaties in de eindknopen) wordt de split gekozen op basis van het minimaliseren van de RSS (Root Summed Square). De variabele die de grootste vermindering in RSS bereikt wordt gekozen als split. De boom splits vervolgend met een top-down greedy aanpak (recursive binary splitting). Het splitsen wordt "greedy" (hebberig) genoemd omdat het algoritme de beste splits zo snel mogelijk probeert uit te voeren, in plaats van deze te bewaren voor later.
Bij classificatie bomen (waar de output wordt bepaald door te kijken naar de observaties in de eindknopen) wordt de split bepaald door een van de volgende methoden:
* Gini Index -- een mate van knoop "puurheid". Hoe kleiner de Gini index, hoe puurder de knoop (hoe meer de observaties op elkaar lijken). Om te splitsen, moet de Gini index van een kindknoop lager zijn dan die van zijn ouder.
* Entropy -- Entropy is een mate van chaos (onpuurheid). Voor een binaire klasse ($a$ of $b$) is de Entropy maximaal als $P = 0.5$; dat wil zeggen, als P$(X=a) = 0.5$ of $P(X=b) = 0.5$, oftewel, de kans dat een nieuwe observatie label $a$ heeft is 50% (idem voor label $b$). De Entropy is minimaal als deze kans 0 of 1 is.
$$Entropy = - p(a)*log(p(a)) - p(b)*log(p(b))$$

![Entropy](http://blog.hackerearth.com/wp-content/uploads/2016/12/ent.png)

In een notendop, elke boom probeert om regels te cree\"eren zodanig dat de resulterende eindknopen zo puur mogelijk zijn. Hoe hoger de puurheid, hoe zekerder de beslissing die wordt genomen.

## Bouwen van een Decision Tree
Doe het volgende:
* Laad de Titanic Data set;
* Bepaal (op het oog) de belangrijkste 3 variabelen waarmee je zou kunnen bepalen of iemand overleeft;
* Implementeer de Entropy-formule van hierboven, en bepaal de Entropy van elk van deze drie variabelen.

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [2]:
# Load Titanic Data set
data = pd.read_csv("data/titanic.csv", delimiter="\t")

In [3]:
data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# Determine best classification variables for Survival
parameters = [
    "sex",
    "Pclass",
    "age"
]

In [5]:
# Implement Entropy
def P(total_len, p):
    return (100/total_len*p)/100

def entropy(pa, pb):
    return -(pa)*np.log(pa) - (pb)*np.log(pb)

# test
survived = data["Survived"]
dead = survived.value_counts()[0]
live = survived.value_counts()[1]
pd = P(len(survived), dead)
pl = P(len(survived), live)

answer = entropy(pd, pl)
answer

In [16]:
# Calculate and show Entropy of selected variables

# je bent een man
sex = data[["Sex", "Survived"]]
male = sex[sex["Sex"]=="male"]
amount_male_life = sex["Survived"].value_counts()[1]
amount_male_dead = sex["Survived"].value_counts()[0]

pa = P(len(sex), amount_male_life)
pb = P(len(sex), amount_male_dead)
print(entropy(pa, pb))

# je reist derde klas
clas = data[["Pclass", "Survived"]]
clas_3 = clas[clas["Pclass"]==3]
amount_clas_3_life = clas_3["Survived"].value_counts()[1]
amount_clas_3_dead = clas_3["Survived"].value_counts()[0]

pa = P(len(clas_3), amount_clas_3_life)
pb = P(len(clas_3), amount_clas_3_dead)
print(entropy(pa, pb))

# je bent jonger dan 18
age = data[["Age", "Survived"]]
teener = age[age["Age"]<18]
amount_teener_life = teener["Survived"].value_counts()[1]
amount_teener_dead = teener["Survived"].value_counts()[0]

pa = P(len(teener), amount_teener_life)
pb = P(len(teener), amount_teener_dead)
print(entropy(pa, pb))


0.645033151676034
0.6036372515003261
0.6791932659915256
0.6730116670092565


In [ ]:
# Bouw een decision tree (https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
# Kiest je sklearn model ook voor dezelfde root-variabele?

## Vervolg
Decision Trees hebben te leiden van hoge variantie ("high variance"). "High Variance" betekent dat het een hoge voorspellingsfout bij ongeziene data. Dit probleem kan worden opgelost door meer data te gebruiken voor het trainen. Maar als je data set beperkt is, kan je gebruik maken van resampling technieken, zoals bagging en random forest om meer data te genereren.

## Hoe werkt het precies? (Random forest)
Het bouwen van meerdere decision trees resulteert in een bos (forest). Een random forest werkt als volgt:
Eerst maakt het gebruik van het Bagging (Bootstrap Aggregating) algoritme om random samples te creeeren. Gegeven een data set D1 ($n$ rijen en $p$ kolommen) creeert het een nieuwe data set (D2) door random $n$ gevallen te samplen. Ongeveer $\frac{1}{3}$ van de rijen van D1 worden achterwege gelaten; deze heten de Out of Bag (OOB) samples.

Vervolgens wordt het model getraind op D2. De OOB samples worden gebruikt om te nauwkeurigheid van dit model te bepalen.
Van de $p$ kolommen worden $P << p$ kolommen geselcteerd in elke knoop. De $P$ kolommen worden random gekozen. Veelal wordt voor regressie gekozen voor een $P=p/3$ en voor classificatie voor $P=sqrt(p)$.

In tegenstelling tot een boom, wordt er bij een random forest geen pruning toegepast; dat wil zeggen, elke boom wordt volledig gegroeid. Bij decision trees wordt pruning gebruikt om overfitting te voorkomen. Pruning betekent dat subtrees gekozen worden die de laagste test error rate hebben. We kunnen cross validation toepassen om de test error te bepalen van subtrees.

Meerdere bomen worden op deze wijze gemaakt, en het uiteindelijke model is verkregen door het nemen van een gemiddelde of majority vote van de resultaten van elke boom.
Elke boom wordt gegroeid op een ander deel van de originele data. Aangezien de OOB gebruikt kan worden voor het bepalen van de error, is cross validatie niet nodig bij het maken van een random forest.


## Bouwen van een Random Forest
Doe het volgende:
* Implementeer het bagging algoritme om verschillende nieuwe data sets te kunnen maken van de originele data set;
* Creeer [50]? decision trees (sklearn trees) op nieuw gegenereerde data sets;
* Valideer je random forest (gebruik majority voting voor ensembling) op een (van te voren) apart gezette validatie set.
* Vergelijk je implementatie/score tegenover de sklearn implementatie: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:
# Implement Bagging Algorithm
def bagging(num_bags):
    bags = []
    for _ in range(num_bags):
        bags.append(data.sample(frac=0.7))
    return bags

In [ ]:
# Create 50 new data sets
bags = bagging(50)

In [ ]:
# Train 50 decision trees on new data sets

In [ ]:
# Implement SKLearn Random Forest on Titanic Data set
# Compare performances

## Bronnen
* [1] Practical Tutorial on Random Forest and Parameter Tuning in R: https://www.hackerearth.com/practice/machine-learning/machine-learning-algorithms/tutorial-random-forest-parameter-tuning-r/tutorial/